In [1]:
import os
import schnetpack as spk
from schnetpack.datasets import QM9
import schnetpack.transform as trn

# https://schnetpack.readthedocs.io/en/latest/tutorials/tutorial_01_preparing_data.html
from schnetpack.data import ASEAtomsData
#from schnetpack.transform import ASENeighborList

import torch
import torchmetrics
import pytorch_lightning as pl

qm9tut = './qm9tut'
if not os.path.exists('qm9tut'):
    os.makedirs(qm9tut)

In [2]:
PROPERTY_LIST = [QM9.homo, QM9.lumo, QM9.zpve, QM9.Cv]
PROPERTY = QM9.lumo

In [3]:
qm9data = QM9(
    './qm9.db',
    batch_size=100,
    num_train=110000,
    num_val=10000,
    transforms=[
        trn.ASENeighborList(cutoff=5.),
        #trn.RemoveOffsets(PROPERTY, remove_mean=True, remove_atomrefs=True),
        trn.CastTo32()
    ],
    #property_units={QM9.U0: 'eV'},
    num_workers=4,
    split_file=os.path.join(qm9tut, "split.npz"),
    pin_memory=False, # set to false, when not using a GPU
    load_properties=[PROPERTY], # load relevant properties
)
qm9data.prepare_data()
qm9data.setup()

In [4]:
print('Number of reference calculations:', len(qm9data.dataset))
print('Number of train data:', len(qm9data.train_dataset))
print('Number of validation data:', len(qm9data.val_dataset))
print('Number of test data:', len(qm9data.test_dataset))
print('Available properties:')

for p in qm9data.dataset.available_properties:
    print('-', p)

Number of reference calculations: 133885
Number of train data: 110000
Number of validation data: 10000
Number of test data: 13885
Available properties:
- rotational_constant_A
- rotational_constant_B
- rotational_constant_C
- dipole_moment
- isotropic_polarizability
- homo
- lumo
- gap
- electronic_spatial_extent
- zpve
- energy_U0
- energy_U
- enthalpy_H
- free_energy
- heat_capacity


In [5]:
example = qm9data.dataset[0]
print('Properties:')

for k, v in example.items():
    print('-', k, ':', v.shape)

Properties:
- _idx : torch.Size([1])
- lumo : torch.Size([1])
- _n_atoms : torch.Size([1])
- _atomic_numbers : torch.Size([5])
- _positions : torch.Size([5, 3])
- _cell : torch.Size([1, 3, 3])
- _pbc : torch.Size([3])


In [6]:
for batch in qm9data.val_dataloader():
    print(batch.keys())
    break


dict_keys(['_idx', 'lumo', '_n_atoms', '_atomic_numbers', '_positions', '_cell', '_pbc', '_idx_i_local', '_idx_j_local', '_offsets', '_idx_m', '_idx_j', '_idx_i'])


In [7]:
""" # OBS - Lidt underligt de er anderledes end i test.ipynb?
atomrefs = qm9data.train_dataset.atomrefs
print('U0 of hyrogen:', atomrefs[QM9.U0][1].item(), 'eV')
print('U0 of carbon:', atomrefs[QM9.U0][6].item(), 'eV')
print('U0 of oxygen:', atomrefs[QM9.U0][8].item(), 'eV') """

" # OBS - Lidt underligt de er anderledes end i test.ipynb?\natomrefs = qm9data.train_dataset.atomrefs\nprint('U0 of hyrogen:', atomrefs[QM9.U0][1].item(), 'eV')\nprint('U0 of carbon:', atomrefs[QM9.U0][6].item(), 'eV')\nprint('U0 of oxygen:', atomrefs[QM9.U0][8].item(), 'eV') "

In [8]:
cutoff = 5.
n_atom_basis = 30

pairwise_distance = spk.atomistic.PairwiseDistances() # calculates pairwise distances between atoms
radial_basis = spk.nn.GaussianRBF(n_rbf=20, cutoff=cutoff)
schnet = spk.representation.SchNet(
    n_atom_basis=n_atom_basis, n_interactions=3,
    radial_basis=radial_basis,
    cutoff_fn=spk.nn.CosineCutoff(cutoff)
)
#output_key = QM9.zpve
pred_property = spk.atomistic.Atomwise(n_in=n_atom_basis, output_key=PROPERTY)

nnpot = spk.model.NeuralNetworkPotential(
    representation=schnet,
    input_modules=[pairwise_distance],
    output_modules=[pred_property],
    postprocessors=[trn.CastTo64()] #, trn.AddOffsets(PROPERTY, add_mean=True, add_atomrefs=True)]
)

In [9]:
output_property = spk.task.ModelOutput(
    name=PROPERTY,
    loss_fn=torch.nn.MSELoss(),
    loss_weight=1.,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

In [10]:
task = spk.task.AtomisticTask(
    model=nnpot,
    outputs=[output_property],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4}
)

/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [11]:
logger = pl.loggers.TensorBoardLogger(save_dir=qm9tut)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(qm9tut, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    default_root_dir=qm9tut,
    max_epochs=3, # for testing, we restrict the number of epochs
)
trainer.fit(task, datamodule=qm9data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 16.4 K
1 | outputs | ModuleList             | 0     
---------------------------------------------------
16.4 K    Trainable params
0         Non-trainable params
16.4 K    Total params
0.066     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 100. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:   4%|▍         | 44/1100 [01:05<26:16,  0.67it/s, v_num=10] 

/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
nnpot.representation.interactions[2].f2out[1] #.out_features

Dense(
  in_features=30, out_features=30, bias=True
  (activation): Identity()
)

### Inference

In [ ]:
import torch
import numpy as np
from ase import Atoms

best_model = torch.load(os.path.join(qm9tut, 'best_inference_model'))

In [ ]:
for batch in qm9data.test_dataloader():
    result = best_model(batch)
    print("Result dictionary:", result)
    break

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [ ]:
best_model.representation.interactions[2].f2out[1].out_features
best_model.representation.interactions[2].f2out[1].weight
best_model.representation.interactions[2].f2out[1].weight.shape

30

torch.Size([30, 30])